<a href="https://colab.research.google.com/github/Mohamed-Harish-M-123/Capstone-Project-IV/blob/main/Online_Retail_Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Project Name: Online Retail Customer Segmentation**
####**Project Type:** Unsupervised ML
####**Contribution:** Individual
####**Name:** MOHAMED HARISH

##**GitHub link:** https://github.com/Mohamed-Harish-M-123/Capstone-Project-IV

##**Business context:**
####In this project, our task is to identify major customer segments on a transactional data set which contains all the transactions occuring between 01/12/2010 and 09/12/2011 for a UK based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customer of the company are wholesalers.